In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K


%matplotlib inline

PARAMETERS

The values used in the base model of transformer were; num_layers=6, d_model = 512, dff = 2048. See the paper for all the other versions of the transformer.


In [2]:
BERT_MODEL_URL = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

SEQ_LEN = 512
MAX_EXAMPLE_LEN = 512

BATCH_SIZE = 1
BUFFER_SIZE = 1

num_layers = 4
d_model = 128
dff = 512
num_heads = 8

# input_vocab_size = tokenizer_pt.vocab_size + 2
# target_vocab_size = tokenizer_en.vocab_size + 2

dropout_rate = 0.1

In [3]:
from data.load import load_cnn_dailymail
from ops.session import initialize_vars


train_dataset, val_dataset, test_dataset, metadata = load_cnn_dailymail()  

Instructions for updating:
Colocations handled automatically by placer.


W0527 14:08:34.972596 140532205602624 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0527 14:08:37.302265 140532205602624 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [4]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run()
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [5]:
from models.transformer import Transformer

In [6]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:

#     sess.run(iter.initializer)
#     tmp = sess.run(el)
#     print(tmp)
    sample_transformer = Transformer(
        num_layers=2, d_model=512, num_heads=8, dff=2048, 
        input_vocab_size=8500, target_vocab_size=8000)

    temp_input = tf.random.uniform((64, 62))
    temp_target = tf.random.uniform((64, 26))

    fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                                   enc_padding_mask=None, 
                                   look_ahead_mask=None,
                                   dec_padding_mask=None)
    initialize_vars(sess)
    
    print(sess.run(fn_out))  # (batch_size, input_seq_len, d_model)    
#     x0, x1, x2, y0, y1, y2 = tmp
#     tmp2 = model.predict([x0, x1, x2])
    

NameError: name 'LayerNormalization' is not defined

In [30]:
sess = tf.InteractiveSession()


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [5]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    """
    The positional encoding vector is added to the embedding vector.
    Embeddings represent a token in a d-dimensional space where tokens
    with similar meaning will be closer to each other.
    But the embeddings do not encode the relative position of words in a sentence.
    
    So after adding the positional encoding, words will be closer to each other
    based on the similarity of their meaning and their position in the sentence,
    in the d-dimensional space.
    
    >>> pos_encoding = positional_encoding(50, 512)
    >>> print (pos_encoding.shape)

    >>> plt.pcolormesh(pos_encoding[0].eval(), cmap='RdBu')
    >>> plt.xlabel('Depth')
    >>> plt.xlim((0, 512))
    >>> plt.ylabel('Position')
    >>> plt.colorbar()
    >>> plt.show()
    """
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    sines = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    cosines = np.cos(angle_rads[:, 1::2])

    pos_encoding = np.concatenate([sines, cosines], axis=-1)

    pos_encoding = pos_encoding[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


def create_padding_mask(seq):
    """
    Mask all the pad tokens in the batch of sequence.
    It ensures that the model does not treat padding as the input.
    The mask indicates where pad value 0 is present:
    it outputs a 1 at those locations, and a 0 otherwise.    
    """
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions so that we can add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    """
    The look-ahead mask is used to mask the future tokens in a sequence.
    In other words, the mask indicates which entries should not be used. 

    This means that to predict the third word, only the first and second word will be used.
    Similarly to predict the fourth word, only the first, second and the third word will be used and so on.
    """
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)




In [6]:
def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(q, k, v, None)
    print ('Attention weights are:')
    print (temp_attn.eval())
    print ('Output is:')
    print (temp_out.eval())

    
temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# This query aligns equally with the first and second key, 
# so their values get averaged.
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
print_out(temp_q, temp_k, temp_v)




Attention weights are:


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [7]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
print(out.shape, attn.shape)

sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512)))

NameError: name 'MultiHeadAttention' is not defined

models

In [29]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, target_vocab_size=8000)

temp_input = tf.random.uniform((64, 62))
temp_target = tf.random.uniform((64, 26))

fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)


TensorShape([Dimension(64), Dimension(26), Dimension(8000)])

### Optimizer

In [37]:
warmup_steps = 4000
initial_lr = 0.0003

def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    '''Noam scheme learning rate decay
    init_lr: initial learning rate. scalar.
    global_step: scalar.
    warmup_steps: scalar. During warmup_steps, learning rate increases
        until it reaches init_lr.
    '''
    step = tf.cast(global_step + 1, dtype=tf.float32)
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)


global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# train_op = optimizer.minimize(loss, global_step=global_step)

In [31]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """
    Use the Adam optimizer with a custom learning rate scheduler according to the formula
    """
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


AttributeError: module 'tensorflow._api.v1.keras.optimizers' has no attribute 'schedules'

### Losses and Metrics

In [43]:
loss_object = tf.keras.losses.sparse_categorical_crossentropy(from_logits=True)


TypeError: sparse_categorical_crossentropy() missing 2 required positional arguments: 'y_true' and 'y_pred'

In [41]:
def loss_function(real, pred):
    """
    Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')


In [44]:
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


In [25]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, target_vocab_size=8000)

output, attn = sample_decoder(tf.random.uniform((64, 26)), 
                              enc_output=sample_encoder_output, 
                              training=False, look_ahead_mask=None, 
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape, attn['decoder_layer2_block1'].shape


(TensorShape([Dimension(64), Dimension(26), Dimension(512)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(62)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(26)]))

In [21]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500)

sample_encoder_output = sample_encoder(tf.random.uniform((64, 62)), 
                                       training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)


(64, 62, 512)


In [18]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output, 
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)


TensorShape([Dimension(64), Dimension(50), Dimension(512)])

In [12]:
class BertLayer(tf.keras.layers.Layer):
    """
    Custom Keras layer, integrating BERT from tf-hub
    """
    def __init__(self, url, seq_len=512, d_embedding=768, n_fine_tune_layers=None, **kwargs):
        self.url = url
        self.n_fine_tune_layers = n_fine_tune_layers
        self.seq_len = seq_len
        self.d_embedding = d_embedding
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.url,
            trainable=self.trainable,
            name="{}_bert_module".format(self.name)
        )

        trainable_vars = self.bert.variables
        
        if self.trainable:

            # Remove unused layers
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)

            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        
        input_ids, input_mask, segment_ids = inputs
        
        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["sequence_output"]
        return result

    def compute_output_shape(self, input_shape):
        raise ValueError()
        return (input_shape[0], self.seq_len, self.d_embedding)


In [14]:
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 12:59:35.651068 140526081922880 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [9]:
config.SEQ_LEN

512